# CARTE-Enbridge Bootcamp
#### Lab 5-2

Dealing with Unlabelled Data

Throughout the week, we have focused on methods for machine learning that rely on predicting a known quantity. That is, all the data we've worked with requires some examples of the factor we want to predict - whether that's the rating of a bottle of wine, or the severity of a car crash.

However, there are often situations where we want to understand data better, but we don't have reliably labelled data. Maybe this is because the label is challenging to acquire (maybe we need people to label our data by hand), or maybe it's just that we don't know what we want to predict yet.

In this final lab, we will run through a number of methods for analyzing data without having clear labels to predict on.

## K-Means Clustering

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from sklearn.datasets import make_blobs
from sklearn.cluster import KMeans

K-Means clustering is an excellent method for uncovering groups within a dataset, without needing to know what those groups are beforehand. The intuition is simple - we are trying to find groups of items that are more similar to each other than they are to items in any other group.

The K-Means algorithm identifies a pre-determined number of clusters (K) within an unlabeled dataset. It does this using a simple procedure describing what each cluster should look like: 

* Each cluster can be represented by the average (mean) of all the points in that cluster.
* Each point is closer to the mean point in its own cluster than to that of any other cluster.

Let's get a visual representation of what this looks like. First we will generate a simple 2D dataset containing four distinct blobs:

In [ ]:
X, y_true = make_blobs(n_samples=300, centers=4,
                       cluster_std=0.60, random_state=0)
plt.scatter(X[:, 0], X[:, 1], s=50);

It's pretty easy to tell where the four clusters are by eye, but let's see how K-Means performs at picking them out. We will also indicate the representative point for each cluster (the _cluster centre_).

In [ ]:
kmeans = KMeans(n_clusters=4)
kmeans.fit(X)
y_kmeans = kmeans.predict(X)

plt.scatter(X[:, 0], X[:, 1], c=y_kmeans, s=50, cmap='viridis')
centers = kmeans.cluster_centers_
plt.scatter(centers[:, 0], centers[:, 1], c='black', s=200, alpha=0.5);

The good news is that the K-Means algorithm (at least in this simple case) assigns the points to clusters very similarly to how we might assign them by eye. But how does K-Means identify these groups? The first thought for many is that we could directly compare every pair of points, and use that information to group them. But doing this requires exponentially more computing for every point we add, just like with our recommendation case yesterday. Instead, we use a clever algorithm called Expectation Maximization (EM) that allows us to make a much smaller number of comparisons, dramatically speeding things up.

In short, K-Means consists of the following procedure:

* Guess some cluster centres (at random, or using a heuristic)
* Repeat until convergence (i.e. the clusters stop changing between steps):
    * Assign each point to the closest cluster centre
    * Re-calculate the average point for each cluster (i.e. its cluster centre)
    
The K-Means algorithm is simple enough that we can write it in a few lines of code. The following is a very basic implementation:

In [ ]:
from sklearn.metrics import pairwise_distances_argmin

def find_clusters(X, n_clusters, rseed=2):
    # 1. Randomly choose clusters
    rng = np.random.RandomState(rseed)
    i = rng.permutation(X.shape[0])[:n_clusters]
    centers = X[i]
    
    while True:
        # 2a. Assign labels based on closest center
        labels = pairwise_distances_argmin(X, centers)
                
        # 2b. Find new centers from means of points
        new_centers = np.array([X[labels == i].mean(0)
                                for i in range(n_clusters)])
        
        # 2c. Check if the clusters changed
        if np.all(centers == new_centers):
            break # If the clusters didn't change, we are finished!
            
        centers = new_centers
        plt.scatter(X[:, 0], X[:, 1], c=labels,
                    s=50, cmap='viridis')
        plt.pause(0.05)
    return centers, labels

centers, labels = find_clusters(X, 4)


## Caveats

If K-Means were perfect, we could stop here. But unfortunately, while elegant, this algorithm does come with its own set of problems. For example, the random seed used to initialize the clusters can affect the results - there is no guarantee that the algorithm will find the best _possible_ solution. (For an example of this, try changing the random seed in the previous example to `3`).

Another common problem with K-Means is that you must tell it how many clusters to look for - it cannot learn this from the data. This is fine in simple, low-dimensional examples such as the one above, but when the data is high-dimensional it becomes impossible to eyeball the correct number of clusters for your data. In fact, we can suggest an obviously incorrect number of clusters, and the model won't tell us that we're doing anything wrong:

In [ ]:
kmeans = KMeans(n_clusters=50)
kmeans.fit(X)
y_kmeans = kmeans.predict(X)

plt.scatter(X[:, 0], X[:, 1], c=y_kmeans, s=25, cmap='nipy_spectral')
plt.colorbar();

Again, this is obviously wrong in this example, but the algorithm has no way of knowing this. However, there *is* a way to identify the best value for K from the data - at least, heuristically.

## Finding the Elbow

If you were tasked with calculating a score that indicates the best possible fit for the data, you might try the average distance for each point in a cluster to its cluster centre. The logic here would be that a better cluster should have all its points close to its centre - hence, the descriptive power of that cluster centre would be higher.

Unfortunately, you may realize, the more clusters there are, the better this score would be. In fact, for N data points, the optimal value would be N=K! At this value, every point would be exactly its cluster centre. But then we aren't really clustering at all!

The point is, as K increases, the average proximity to the cluster centre is _guaranteed_ to increase. But there is still some information we can gain from this metric. While the value itself always increases, the _rate of change_ can differ - and here is the secret to identifying the best value for K.

In [ ]:
from scipy.spatial.distance import cdist

av_dist = []

for k in range(1,10):
    km = KMeans(n_clusters=k).fit(X)
    av_dist.append(
        sum(
            np.min(
                cdist(X, km.cluster_centers_, 'euclidean'),
                axis=1
            )
        ) / X.shape[0]
    )
    
plt.plot(range(1,10), av_dist)
plt.xlabel('Value for K')
plt.ylabel('Average distance to cluster centre');

Looking at this graph, it's transparently obvious where the improvement with a higher K stops becoming substantial - K = 4, just as we would expect. If we plot the _improvement_, this transition becomes even more obvious:

In [ ]:
plt.plot(range(1,10),-np.gradient(av_dist))
plt.xlabel('Value for K')
plt.ylabel('Improvement from previous K');

The _elbow_ is the point in the line where improvement transitions from substantial to minimal. Of course, such a clear elbow is not guaranteed - if your dataset doesn't cluster very well, it might be the case that there's no clear transition from good improvement to poor improvement. But in situations where there *is* a good value for K waiting to be found, the Elbow method can help us consistently find it.

### Dimensionality reduction and principle component analysis

When working with data in the real world, we're almost always dealing with more features than we can conveniently plot in a single figure. But sometimes, we want to be able to quickly see relationships or patterns across all of our data in a single go. This is where dimensionality reduction can come in. "Dimensionality Reduction" is exactly what it sounds like - reducing the number of dimensions (aka. features) so that we can more easily look at our data.

Obviously, as suggested above, one reason to do this is to aid visualization. But that's far from the only reason dimensionality reduction is useful! These techniques also allow us to filter noise, extract useful features, and much more.

Let's dive into PCA. We'll first take a two dimensional dataset and reduce it to one dimension.

In [ ]:
rng = np.random.RandomState(1)

X = np.dot(rng.rand(2,2),
          rng.randn(2,200)).T
plt.scatter(X[:,0],X[:,1])
plt.axis('equal');

Here we have a two dimensional dataset that, by eye, has a nearly linear relationship between the X and Y values. This is reminiscent of the type of data we would look to explore with linear regression, but the problem setting here is slightly different: rather than attempting to predict the y values from the x values, the unsupervised learning problem attempts to learn about the relationship between the x and y values.

In principal component analysis, this relationship is quantified by finding a list of the principal axes in the data, and using those axes to describe the dataset. Using Scikit-Learn's PCA estimator, we can compute this as follows:

In [ ]:
from sklearn.decomposition import PCA
pca = PCA(n_components=2)
pca.fit(X);

The fit learns some quantities from the data, most importantly the "components" and "explained variance":

In [ ]:
pca.components_

In [ ]:
pca.explained_variance_

To see what these numbers mean, let's visualize them as vectors over the input data, using the "components" to define the direction of the vector, and the "explained variance" to define the squared-length of the vector:

In [ ]:
def draw_vector(v0, v1, ax=None):
    ax = ax or plt.gca()
    arrowprops=dict(arrowstyle='->',
                    linewidth=2,
                    shrinkA=0, shrinkB=0, color='black')
    ax.annotate('', v1, v0, arrowprops=arrowprops)

# plot data
plt.scatter(X[:, 0], X[:, 1], alpha=0.2)
for length, vector in zip(pca.explained_variance_, pca.components_):
    v = vector * 3 * np.sqrt(length)
    draw_vector(pca.mean_, pca.mean_ + v)
plt.axis('equal');

These vectors represent the principal axes of the data, and the length of the vector is an indication of how "important" that axis is in describing the distribution of the data—more precisely, it is a measure of the variance of the data when projected onto that axis. The projection of each data point onto the principal axes are the "principal components" of the data.

You have likely noticed that in this example, almost all of the variance is explained by the first axis, while the second axis explains very little variance. This is where PCA becomes a tool for dimensionality reduction. By transforming the data to lie along the PCA dimensions and then only keeping the first dimension, we can reduce our dataset into one-dimensional space:

In [ ]:
pca = PCA(n_components=1)
pca.fit(X)
X_pca = pca.transform(X)
print("original shape:   ", X.shape)
print("transformed shape:", X_pca.shape)
plt.hist(X_pca,bins=20)
plt.xlabel('Value in reduced space')
plt.ylabel('Count');

If we project our new, low-dimensional data back into the original space, we can get a good visual intuition of what PCA has done for us:

In [ ]:
X_new = pca.inverse_transform(X_pca)
plt.scatter(X[:, 0], X[:, 1], alpha=0.2)
plt.scatter(X_new[:, 0], X_new[:, 1], alpha=0.8)
plt.axis('equal');

The light points are the original data, while the orange points are the projected version. This makes clear what a PCA dimensionality reduction means: the information along the least important principal axis or axes is removed, leaving only the component(s) of the data with the highest variance. The fraction of variance that is cut out (proportional to the spread of points about the line formed in this figure) is roughly a measure of how much "information" is discarded in this reduction of dimensionality.

This reduced-dimension dataset is in some senses "good enough" to encode the most important relationships between the points: despite reducing the dimension of the data by 50%, the overall relationship between the data points are mostly preserved.

The usefulness of the dimensionality reduction may not be entirely apparent in only two dimensions, but becomes much more clear when looking at high-dimensional data. To see this, let's take a quick look at the application of PCA to a much higher dimensional dataset.



In [ ]:
from sklearn.datasets import load_digits
digits = load_digits()
digits.keys()
# set up the figure
fig = plt.figure(figsize=(6, 6))  # figure size in inches
fig.subplots_adjust(left=0, right=1, bottom=0, top=1, hspace=0.05, wspace=0.05)

# plot the digits: each image is 8x8 pixels
for i in range(64):
    ax = fig.add_subplot(8, 8, i + 1, xticks=[], yticks=[])
    ax.imshow(digits.images[i], cmap=plt.cm.binary, interpolation='nearest')
    
    # label the image with the target value
    ax.text(0, 7, str(digits.target[i]))

This dataset contains a series of hand-written digits, translated to an 8x8 (i.e. 64 dimensional) grid. This is similar to the MNIST data we worked with previously, except it starts out with even fewer pixels. To gain some intuition into the relationships between these points, we can use PCA to project them to a more manageable number of dimensions, say two:

In [ ]:
pca = PCA(2)  # project from 64 to 2 dimensions
projected = pca.fit_transform(digits.data)
print(digits.data.shape)
print(projected.shape)

In [ ]:
plt.scatter(projected[:, 0], projected[:, 1],
            c=digits.target, edgecolor='none', alpha=0.5,
            cmap=plt.cm.get_cmap('nipy_spectral', 10))
plt.xlabel('component 1')
plt.ylabel('component 2')
plt.colorbar();

Recall what these components mean: the full data is a 64-dimensional point cloud, and these points are the projection of each data point along the directions with the largest variance. Essentially, we have found the optimal stretch and rotation in 64-dimensional space that allows us to see the layout of the digits in two dimensions, and have done this in an unsupervised manner—that is, without reference to the labels.

By looking at this figure, we can begin to understand the ways that some digits are more or less similar to each other. For example, it makes some sense that we see significant overlap between 7 and 1 - which can often be drawn in a simliar way to each other.

## TSNE

We are now going to look at a different method of dimensionality reduction: t-SNE, a method partially developed here at U of T by Geoffrey Hinton in 2008. It differs from PCA in that it uses the _local relationships_ between points to create a low-dimensional mapping. Among other things, this allows t-SNE to capture non-linear structures in the data.

There are a number of other benefits to t-SNE over PCA, but since we are focusing on using dimensionality reduction for visualization we will stick to the benefits which are apparent in that space. Because t-SNE considers the local relationships between points, it ensures that the distances between points in the low dimensional mapping are representative of the distances in the original space. This makes it a lot more useful for visualization compared to PCA!

### How t-SNE works

t-SNE – at a high level – basically works like this:

Step 1: In the high-dimensional space, create a probability distribution that dictates the relationships between various neighboring points

Step 2: It then tries to recreate a low dimensional space that follows that probability distribution as best as possible.

The “t” in t-SNE comes from the t-distribution, which is the distribution used in Step 2. The “S” and “N” (“stochastic” and “neighbor”) come from the fact that it uses a probability distribution across neighboring points.

Let's swap out our smaller version of the dataset for MNIST, so that we can reduce even more dimensions:

In [ ]:
from sklearn.datasets import fetch_openml
mnist = fetch_openml('mnist_784',cache=False) #This could take a little while - it's a lot of data.
X = mnist['data']
y = mnist['target']
y = [int(t) for t in y]
X.shape

First, we use our knowledge of PCA to reduce MNIST to two dimensions and plot it, color coded by target value.

In [ ]:
plt.figure(figsize=(16,10));

pca = PCA(2)  # project from 64 to 2 dimensions
projected = pca.fit_transform(X)

plt.scatter(projected[:, 0], projected[:, 1],
            c=y, edgecolor='none', alpha=0.5,
            cmap=plt.cm.get_cmap('nipy_spectral', 10))
plt.xlabel('component 1')
plt.ylabel('component 2')
plt.colorbar();

Okay, so that should be similar to what we saw above with the smaller digit dataset, but it's honestly not very useful. Everything is just in a huge, formless blob and we aren't really gathering much information - it's just overwhelming. So let's see what t-SNE can do. 

**NOTE** t-SNE is going to take a lot longer to run than PCA. Although you *can* run it on the entire 70,000 image dataset, this could take a significant amount of time even on a high end computer. If you don't want to wait around that long, uncomment the line below to get a random subsample of the data which you can use instead.

In [ ]:
from sklearn.model_selection import train_test_split
_,X,_,y = train_test_split(X,y,test_size=1/14)

In [ ]:
import time
from sklearn.manifold import TSNE

time_start = time.time()
tsne = TSNE(n_components=2, verbose=1, perplexity=40, n_iter=300)
tsne_results = tsne.fit_transform(X)
print('t-SNE done! Time elapsed: {} seconds'.format(time.time()-time_start))

In [ ]:
plt.figure(figsize=(16,10))
plt.scatter(tsne_results[:, 0], tsne_results[:, 1],
            c=y, edgecolor='none', alpha=0.5,
            cmap=plt.cm.get_cmap('nipy_spectral', 10))
plt.xlabel('component 1')
plt.ylabel('component 2')
plt.colorbar();

Woah! Using t-SNE, the clusters are much more clearly defined, with different numbers being clearly separated in the low dimensional space. Running a clustering algorithm like K-Means now would likely give us pretty reasonable performance on classifying the digits, even just from two dimensions.